In [21]:
from copypastelrm.utils.json_tools import read_jsonl_to_list, read_json, save_jsonl
from copypastelrm.metrics.HotpotQA import compute_answer_em_hit_f1
from copypastelrm.metrics.utils import extract_answer_and_facts, extract_answer_and_facts_old

In [11]:
path_1 = "key_data/pass_at_k_equal_0_subset/hotpotqa_sampling128_answerf1_tips_at_120_step.jsonl"
path_2 = "key_data/pass_at_k_equal_0_subset/hotpotqa_samping42_answer_em_tips_at_32_step.jsonl"

paths = [
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_copypaste-1766487293.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_copypaste-1766590282.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_deepseek-1766478707.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_deepseek-1766583151.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_ircot-1766471827.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_rag-1766424839.json",
]

In [18]:
correct_ids = set()

for path in paths:
    data = read_json(path)['data']
    info = read_json(path)['info']
    prompt_type = info['prompt_type']
    for sample_id, item in data.items():
        dataset = item['dataset']
        if 'hotpotqa' not in dataset:
            continue

        predict = item['predict']
        if prompt_type in ['copypaste', 'deepseek']:
            predict_answer, _ = extract_answer_and_facts_old(predict)
        else:
            predict_answer, _ = extract_answer_and_facts(predict)
        
        answer = item['answer']

        em, hit, f1 = compute_answer_em_hit_f1(predict_answer, answer)

        if f1 > 0:
            correct_ids.add(sample_id)

len(correct_ids)


6830

In [15]:
data_1 = set(read_jsonl_to_list(path_1))
data_2 = set(read_jsonl_to_list(path_2))

len(data_1), len(data_2), len(data_1 & data_2)

(2919, 2583, 2317)

In [19]:
# 求并集
data_union = data_1 & data_2

len(data_union)



2317

In [22]:
real_wrong_ids = data_union - correct_ids
len(real_wrong_ids)

465

In [23]:
save_jsonl(list(real_wrong_ids), "key_data/hotpotqa_hard.jsonl")